<a href="https://colab.research.google.com/github/krislars/R-uncertainty/blob/master/MonteCarloModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import numpy.random as npRand
from matplotlib import pyplot as plt
import scipy.stats as stats

from ipywidgets import interact
from ipywidgets.widgets import FloatSlider

In [2]:
%matplotlib inline

In [108]:
#Defining some parameters:
numVals = 1000
diffABmin = 0.1
diffABmax = 1.9
numSteps = 100

In [109]:
#MultiDimensional linspace
def ndlinspace(start, end, steps):
    if (start.shape != end.shape):
        print("Arrays must be same size")
        return
    if (start.ndim == 1):
        result = np.array(
            [np.linspace(s, e, steps) for s,e in zip(start, end)]
        )
        return result
    result = np.array(
        [ndlinspace(s, e, steps) for s,e in zip(start, end)]
    )
    return result

Data file format is intrinsic stellar H-K, sigma H-K, J-H, sigma J-H ... 
Davenport, J. R., Ivezic, Z., Becker, A. C., Ruan, J. J., Hunt-Walker, N. M., Covey, K. R., & Lewis, A. R. (2014, June). The SDSS-2MASS-WISE 10-dimensional stellar colour locus [Electronic version]. MNRAS, 440(4), 3430-3438.

$$
R = \frac{A-B}{C-D}
$$

We will choose values such that the numerator is diffAB.  So, $A=(diffAB)+B$.  We also choose the value of R to be 1.6, so $C-D=(diffAB)/1.6$.  Therefore, $C = (diffAB)/1.6+D$.

In [110]:
starData = np.loadtxt("https://raw.githubusercontent.com/krislars/R-uncertainty/master/Astro%20Lab%20Star%20Data.txt", unpack=True)

B, dB = starData[0], starData[1]
D, dD = starData[2], starData[3]

diffAB = np.linspace(diffABmin, diffABmax, numSteps)
diffAB = np.transpose(ndlinspace(diffAB, diffAB, 50))

B, dB = ndlinspace(B,B,numSteps), ndlinspace(dB,dB,numSteps)
D, dD = ndlinspace(D,D,numSteps), ndlinspace(dD,dD,numSteps)

A = diffAB + B
C = diffAB/1.6 + D

dA = dC = np.linspace(0.02, 0.02, 50)
dA = dC = ndlinspace(dA, dA, numSteps)

This next line is propagation of error in an arithmetic function.

In [111]:
#calculating theoretical dR
dR = 1.6 *((dB**2 + dA**2)/diffAB**2 + (1.6/diffAB)**2 *(dD**2 + dC**2))**0.5

This function takes a vector of values and a vector of associated uncertainties and returns an array of n = numVals samples selected randomly from a normal  probability distribution.

In [112]:
#choosing random data sets for each star type
def ndNormalData(mean, sigma, numVals):
    if (mean.shape != sigma.shape):
        print("Arrays must be same size")
        return
    if (mean.ndim == 1):
        data = np.array(
            [npRand.normal(m, s, numVals) for m, s in zip(mean, sigma)]
        )
        return data
    
    data = np.array(
        [ndNormalData(m, s, numVals) for m, s in zip(mean, sigma)]
    )
    return data

In [113]:
B_vals = ndNormalData(B, dB, numVals)
D_vals = ndNormalData(D, dD, numVals)
A_vals = ndNormalData(A, dA, numVals)
C_vals = ndNormalData(C, dC, numVals)

In [114]:
#calculating the "Monte Carlo" values for R and dR
R_vals = (A_vals - B_vals) / (C_vals - D_vals)
R_vals.shape

(50, 100, 1000)

Now, we can test the distributions of R.

In [115]:
R_mc = np.mean(R_vals, axis=2)
dR_mc = np.std(R_vals, axis=2)

In [116]:
diffABinit = (diffABmin + diffABmax) /2

def findIndex(diffABval):
    step = (diffABmax - diffABmin) / (numSteps-1)
    result = (diffABval - diffABmin) / step
    return int(result)

def plot_data(diffABval):
    x = np.arange(50)
    fig = plt.figure(figsize=(8, 5))

    red_ax = plt.axes([0.1, 0.2, 0.8, 0.65])
    
    plt.ylim([1,3])
    plt.title('Mean Reddening Across Star Types')
    plt.errorbar(x, R_mc[x, findIndex(diffABval)], yerr=dR_mc[x, findIndex(diffABval)], fmt='ro')
    plt.errorbar(x, np.linspace(1.6,1.6,50), fmt="b--")
    plt.show()
    
interact(plot_data, diffABval=FloatSlider(diffABinit, min=diffABmin, max=diffABmax, step=0.05))


interactive(children=(FloatSlider(value=1.55, description='diffABval', max=3.0, min=0.1, step=0.05), Output())…

<function __main__.plot_data(diffABval)>

Ideas going forward:
* Calculate the uncertainty in the mean.
* Try a weighted mean



In [81]:
#Defines binning of data to evenly space bins close to the mean
#and group the outer bins together. This avoids an expected bincount of 0
#for calculating chi-squared.
def customBinning(data, numBins=20, dataWidth=8):
    low = dataMin = min(data)
    high = dataMax = max(data)
    mean = np.mean(data)
    stdev = np.std(data)
    
    if (dataMin < mean - dataWidth * stdev):
        numBins = numBins - 1
        low = mean - dataWidth * stdev
    if (dataMax > mean + dataWidth * stdev):
        numBins = numBins - 1
        high = mean + dataWidth * stdev
    
    bins = np.linspace(low, high, numBins + 1)
    
    if (dataMin != low):
        bins = np.concatenate((dataMin, bins), axis = None)
    if (dataMax != high):
        bins = np.concatenate((bins, dataMax), axis = None)
    
    return bins

def ndHistogramCounts(Vals, numBins):
    if (Vals.ndim == 1):
        bins = customBinning(Vals, numBins)
        counts, Bins = np.histogram(Vals, bins)
        return counts
    
    results = np.stack(
        ndHistogramCounts(vals, numBins) for vals in Vals
    )
    
    return results

def ndHistogramBins(Vals, numBins):
    if (Vals.ndim == 1):
        bins = customBinning(Vals, numBins)
        #counts, Bins = np.histogram(Vals, bins)
        return bins
    
    results = np.stack(
        ndHistogramBins(vals, numBins) for vals in Vals
    )
    
    return results

def ndNormalCounts(mean, sigma, Bins):
    if (mean.shape != sigma.shape):
        print("Arrays must be same shape")
        return
    
    if (mean.ndim == 1):
        counts = np.diff(np.stack(
            stats.norm.cdf(bins, m, s) for bins, m, s in zip(Bins, mean, sigma)
        ))
        return counts
    
    counts = np.stack(
        ndNormalCounts(m, s, bins) for m, s, bins in zip(mean, sigma, Bins)
    )
    return counts


In [83]:
#Next I need to test "Goodness of fit"
numBins = 20
counts = ndHistogramCounts(R_vals, numBins)
Bins = ndHistogramBins(R_vals, numBins)

#calc expected with stats cdf and np.diff
exp = ndNormalCounts(R_mc, dR_mc, Bins)

#scale expected percentages by sample size
exp = numVals * exp

#Now calculate the Chi^2 values:
chi2 = np.sum(( (counts - exp)**2 / exp), axis=2)

In [84]:
#Constraints in this case are the calculated mean, stdev, and total number of counts
c = 3
d = numBins - c
red_chi2 = chi2/d